In [1]:
!pip install python-chess
!pip install psutil
!pip install google-cloud-storage

import numpy as np
import pandas as pd
import glob
import os
import chess


In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()


In [ ]:
import psutil
memory = psutil.virtual_memory()
print(f"Total RAM: {memory.total / (1024 ** 3):.2f} GB")
!nvidia-smi


Total RAM: 204.46 GB
Sat Oct  7 16:25:24 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              11W /  70W |      5MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
from google.cloud import storage

def download_from_bucket(blob_name, path_to_save_to, bucket_name):
    client = storage.Client()

    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(path_to_save_to)
    print(f"File {blob_name} downloaded to {path_to_save_to}.")

# Use the function to download your file:
download_from_bucket("final_4M_bits.npy", "/content/final_4M_bits.npy", "chess_data_all")
download_from_bucket("final_4M_labels.npy", "/content/final_4M_labels.npy", "chess_data_all")


File final_4M_bits.npy downloaded to /content/final_4M_bits.npy.
File final_4M_labels.npy downloaded to /content/final_4M_labels.npy.


In [2]:
bitboard_dataset=np.load("/content/final_4M_bits.npy")
labels=np.load("/content/final_4M_labels.npy")

In [ ]:
print(bitboard_dataset.shape ,bitboard_dataset[0])
print(labels.shape ,labels[0])

(3952040, 16, 8, 8) [[[  0.   0.   0. ...   0.   0.   0.]
  [ 11.  11.  11. ...  11.  11.  11.]
  [  1.   1.   1. ...   1.   1.   1.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  ...
  [ -1.  -1.  -1. ...  -1.  -1.  -1.]
  [-11. -11. -11. ... -11. -11. -11.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0.  44.   0. ...   0.  44.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  4.   0.   4. ...   4.   0.   4.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 ...

 [[ 55.  44.  33. ...  33.  44.  55.]
  [ 11.  11.  11. ...  11.  11.  11.]
  [  0.   0.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0.   0.   0. ...   0.   0

In [3]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D


# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.2, random_state=42)


# Input layer
input_layer_2d = layers.Input(shape=(16,8, 8))  # for 2D convolutions
input_layer_3d = layers.Reshape((16, 8, 8, 1))(input_layer_2d)  # for 3D convolutions


In [4]:
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation

# Branch 1: Depthwise Convolution
branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
branch1 = BatchNormalization()(branch1)
branch1 = Dropout(0.1)(branch1)


branch4 = Conv2D(filters=3, kernel_size=(6, 6), activation=relu)(branch1)
branch4 = BatchNormalization()(branch4)
branch4 = Dropout(0.1)(branch4)
# branch4 = Conv2D(filters=5, kernel_size=(2, 2), activation=relu)(branch4)

# Branch 2: Conv3D
branch2 = Conv3D(filters=6, kernel_size=(6, 6, 6), padding='valid', activation=relu)(input_layer_3d)
branch2 = BatchNormalization()(branch2)
branch2 = Dropout(0.1)(branch2)
branch2 = Conv3D(filters=3, kernel_size=(2, 2, 2), padding='valid', activation=relu)(branch2)
branch2 = BatchNormalization()(branch2)



# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(branch3_input)
branch3 = Conv2D(filters=6, kernel_size=(5,5), activation=relu)(branch3)
# branch3 = Conv2D(filters=3, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = BatchNormalization()(branch3)
branch3 = Dropout(0.1)(branch3)


branch1 = Flatten()(branch1)
branch2 = Flatten()(branch2)  # Flatten branch2
branch4 = Flatten()(branch4)
branch3 = Flatten()(branch3)  # Flatten branch3

concat = Concatenate()([branch1,branch4, branch2, branch3])

# Dense layers
dense1 = Dense(800)(concat)
dense1 = BatchNormalization()(dense1)
dense1 = Activation('relu')(dense1)
dense1 = Dropout(0.2)(dense1)

dense2 = Dense(400)(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Activation('relu')(dense2)
dense2 = Dropout(0.2)(dense2)

dense3 = Dense(120)(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Activation('relu')(dense3)
dense3 = Dropout(0.2)(dense3)

dense4 = Dense(50)(dense3)
dense4 = BatchNormalization()(dense4)
dense4 = Activation('relu')(dense4)
dense4 = Dropout(0.2)(dense4)

output_layer = Dense(1)(dense4)

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=300, batch_size=400, validation_split=0.1)

# Evaluate the model
loss, mae= model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)

Epoch 1/300
7114/7114 [==============================] - 86s 11ms/step - loss: 8813.6436 - mae: 38.7528 - val_loss: 7973.2393 - val_mae: 36.2919
Epoch 2/300
7114/7114 [==============================] - 72s 10ms/step - loss: 7994.9854 - mae: 37.3245 - val_loss: 8697.3428 - val_mae: 39.5504
Epoch 3/300
7114/7114 [==============================] - 72s 10ms/step - loss: 7544.2451 - mae: 36.4609 - val_loss: 7152.1411 - val_mae: 33.8288
Epoch 4/300
7114/7114 [==============================] - 72s 10ms/step - loss: 7138.2651 - mae: 35.8202 - val_loss: 7197.7998 - val_mae: 34.5941
Epoch 5/300
7114/7114 [==============================] - 72s 10ms/step - loss: 6757.5215 - mae: 35.1530 - val_loss: 6602.5278 - val_mae: 32.5224
Epoch 6/300
7114/7114 [==============================] - 72s 10ms/step - loss: 6451.5669 - mae: 34.5951 - val_loss: 6456.6782 - val_mae: 32.2436
Epoch 7/300
7114/7114 [==============================] - 72s 10ms/step - loss: 6148.6763 - mae: 34.0885 - val_loss: 6368.2158 - va

KeyboardInterrupt: ignored

In [ ]:
model.save("model.h5")
